## Подключение к PT NAD

ptnad-client позволяет выполнять запросы к системе с использованием различных способов аутентификации.

### Локальное подключение

Для быстрого тестирования и отладки можно использовать локальное подключение. В качестве дефолтных значений используется логин "administrator" и пароль "Administr@t0r".

In [59]:
from ptnad import PTNADClient

# Подключение к локальному инстансу PT NAD (пример для тестовой среды)
client = PTNADClient("https://ptlab-nad.ptlab.ptsecurity.ru", verify_ssl=False)

# Установка логина и пароля
client.set_auth(username="administrator", password="Administr@t0r")

# Подключаемся к сервису
client.login()


### Подключение через SSO

Для подключения через SSO необходимо использовать auth_type="sso" и параметры, которые можно получить из файла /opt/ptsecurity/etc/iam_cookie.json на сервере PT NAD.

Пример структуры, получаемой при выводе содержимого "iam_cookie.json":

```bash
cat /opt/ptsecurity/etc/iam_cookie.json
```

```json
{
    "iam": {
        "id": "your_id",
        "name": "PTLAB NAD",
        "redirect_url": "https://ptlab-nad.ptlab.ptsecurity.ru",
        "secret": "your_secret",
        "token": "your_token",
        "token_type": "reference",
        "url": "https://ptlab-core.ptlab.ptsecurity.ru:3334",
        "verify": false
    },
    "tms": {
        "app_id": "your_app_id",
        "display_name": "PTLAB NAD",
        "endpoint": "https://ptlab-nad.ptlab.ptsecurity.ru",
        "request_id": "your_request_id",
        "token": "your_token",
        "url": "https://ptlab-core.ptlab.ptsecurity.ru:3334"
    }
}
```

Для подключения, необходимо взять информацию из ключей "app_id", "secret" и "url".
В качестве auth_type указываем "sso", добавляем свои логин/пароль и оставшиеся поля из iam_cookie.json, где
- client_id = id
- client_secret = secret
- sso_url = url

По итогу получаем:

```python
client = PTNADClient("https://ptlab-nad.ptlab.ptsecurity.ru", verify_ssl=False)
client.set_auth(
    auth_type="sso",
    username="user",
    password="pass",
    client_id="your_id",
    client_secret="your_secret",
    sso_url="https://ptlab-core.ptlab.ptsecurity.ru:3334"
)
client.login()
```

Ссылка на код, где можно подробнее изучить механизмы авторизации: [Code Reference](https://reversenant.github.io/ptnad-client-test/reference/)

### Примеры работы с библиотекой

#### BQL-запросы

В этом примере демонстрируется работа с кастомными фильтрами, хранящимися в файле "docs\data\pilot_filters.json", и выполнение BQL-запросов к PT NAD. Для примера выберем только два поля: IP-адрес источника (src.ip) и IP-адрес назначения (dst.ip). Также ограничим вывод фильтров - по одному на каждую категорию, по 3 события:
- Локально подключаемся PT NAD.
- Загружаем фильтры из файла pilot_filters.json, где каждый фильтр содержит BQL-фрагмент, имя и категорию (есть возможность выбора языка).
- Формируем базовый запрос с временным диапазоном (последние 3 дня).
- Объединяем базовый запрос с фильтрами.
- Группируем запросы по категориям.
- Выполняем каждый запрос и выводим результаты с указанием:
    - категории фильтра;
    - его названия;
    - полного BQL-запроса;
    - результата.

Более подробно про возможные параметры фильтрации и их значения можно почитать тут - [PT NAD Справка](https://help.ptsecurity.com/ru-RU/projects/nad/12.2/help/7083891595)

In [72]:
import json
from datetime import datetime, timedelta
from ptnad import PTNADClient

client = PTNADClient("https://ptlab-nad.ptlab.ptsecurity.ru", verify_ssl=False)
client.set_auth(username="administrator", password="Administr@t0r")
client.login()

with open("../data/pilot_filters.json", "r", encoding="utf-8") as f:
    data = json.load(f)

now = datetime.now()
start_time = (now - timedelta(days=3)).strftime("%Y-%m-%dT%H:%M:%S")
end_time = now.strftime("%Y-%m-%dT%H:%M:%S")

base_query = f"SELECT src.ip, dst.ip FROM flow WHERE start > '{start_time}' AND end < '{end_time}'"

grouped_queries = {}

for item in data["filters"]:
    category = item["category"]["ru"]
    name = item["name"]["ru"]
    filter_query = item["filter"].strip()

    if filter_query:
        full_query = f"{base_query} AND ({filter_query})"
        if category not in grouped_queries:
            grouped_queries[category] = []
        grouped_queries[category].append((name, full_query))

for category, filters in grouped_queries.items():
    print(f"\n▶️ Категория: {category}")
    for name, query in filters[:1]:
        result = client.bql.execute(query)
        print(f"\n🔹 {name}\n{query}")
        print("Results:")
        if isinstance(result, list) and result:
            for i, row in enumerate(result[:3], 1):
                print(f"{i}. {row}")
        else:
            print("Нет результатов.")


▶️ Категория: Протоколы

🔹 Telnet
SELECT src.ip, dst.ip FROM flow WHERE start > '2025-04-07T12:44:18' AND end < '2025-04-10T12:44:18' AND (app_proto == 'telnet')
Results:
Нет результатов.

▶️ Категория: Данные

🔹 Сессии с файлами
SELECT src.ip, dst.ip FROM flow WHERE start > '2025-04-07T12:44:18' AND end < '2025-04-10T12:44:18' AND (files)
Results:
1. ['10.125.9.10', '10.125.136.66']
2. ['10.125.143.4', '23.210.252.238']
3. ['10.125.9.10', '10.125.136.66']

▶️ Категория: Анонимность

🔹 OpenVPN
SELECT src.ip, dst.ip FROM flow WHERE start > '2025-04-07T12:44:18' AND end < '2025-04-10T12:44:18' AND (app_proto == openvpn)
Results:
Нет результатов.

▶️ Категория: Сеть

🔹 Трафик без доменного имени
SELECT src.ip, dst.ip FROM flow WHERE start > '2025-04-07T12:44:18' AND end < '2025-04-10T12:44:18' AND (dst.groups != 'HOME_NET' && !dst.dns)
Results:
1. ['10.125.101.6', '10.158.2.22']
2. ['10.125.126.5', '10.158.6.143']
3. ['10.125.101.3', '10.158.2.22']

▶️ Категория: Веб

🔹 Digital Ocean
SEL

#### Monitoring

Получение общего статуса системы:

In [3]:
from ptnad import PTNADClient
from ptnad.api.monitoring import MonitoringAPI

client = PTNADClient("https://ptlab-nad.ptlab.ptsecurity.ru", verify_ssl=False)
client.set_auth(username="administrator", password="Administr@t0r")
client.login()

monitoring = MonitoringAPI(client)

status = monitoring.get_status()

print(f"Статус: {status.status}")
for problem in status.problems:
    print(f"{problem}")

Статус: green


Получение списка всех триггеров

In [ ]:
from ptnad import PTNADClient
from ptnad.api.monitoring import MonitoringAPI

client = PTNADClient("https://ptlab-nad.ptlab.ptsecurity.ru", verify_ssl=False)
client.set_auth(username="administrator", password="Administr@t0r")
client.login()

monitoring = MonitoringAPI(client)

triggers = monitoring.get_triggers()

# Для удобства установлено ограничение на вывод 5 триггеров
for trig in triggers[:5]:
    print(f"\nTrigger ID: {trig.id}")
    print(f"Тип: {trig.type}")
    print(f"Статус: {trig.status}")
    print(f"Последнее обновление: {trig.updated}")


Trigger ID: updates[PTSecurity]
Тип: updates
Статус: green
Последнее обновление: 2025-04-10T11:17:41.667952Z

Trigger ID: ptdpi-broker_unavailable
Тип: stats
Статус: green
Последнее обновление: 2025-04-10T11:26:22.275682Z

Trigger ID: ptdpi-worker@es_unavailable
Тип: stats
Статус: green
Последнее обновление: 2025-04-10T11:26:22.275846Z

Trigger ID: ptdpi-worker@dns_unavailable
Тип: stats
Статус: green
Последнее обновление: 2025-04-10T11:26:22.275899Z

Trigger ID: nad-reporter_unavailable
Тип: stats
Статус: green
Последнее обновление: 2025-04-10T11:26:22.275953Z


#### Replists

Получение всех списков

In [75]:
from ptnad import PTNADClient
from ptnad.api.replists import RepListsAPI

client = PTNADClient("https://ptlab-nad.ptlab.ptsecurity.ru", verify_ssl=False)
client.set_auth(username="administrator", password="Administr@t0r")
client.login()

replists = RepListsAPI(client)

lists = replists.get_all_lists()
print(f"Общее количество: {len(lists)}")

# Для удобства установлено ограничение на вывод 15 списков
for r in lists[:15]:
    print(f"- [{r['id']}] {r['name']} ({r['type']}, items: {r['items_count']})")


Общее количество: 954
- [24858] ESC-manual-mekotio-ip (ip, items: 1)
- [24311] ESC-auto-spyder-ip (ip, items: 1)
- [25318] ESC-auto-mispadu-ip (ip, items: 1)
- [25984] ESC-auto-apt35-ip (ip, items: 2)
- [14896] malware (md5, items: 2856)
- [23653] ESC-auto-apt28-ip (ip, items: 2)
- [23000] ESC-manual-gsocket-ip (ip, items: 6)
- [26133] ESC-manual-pocorat-ip (ip, items: 1)
- [24067] ESC-manual-bandook-ip (ip, items: 1)
- [23001] ESC-manual-stickywerewolf-darktrack-ip (ip, items: 1)
- [23002] ESC-auto-stargazer_goblin-ip (ip, items: 1)
- [25141] ESC-auto-chamelgang-ip (ip, items: 1)
- [25153] ESC-auto-darkcomet-ip (ip, items: 1)
- [25075] ESC-auto-warmcookie-ip (ip, items: 1)
- [2118] ESC-auto-cloudeye-ip (ip, items: 19)


Поиск по ключевому слову

In [76]:
results = replists.get_lists(search="APT")

# Для удобства установлено ограничение на вывод 15 элементов
for r in results[:15]:
    print(f"{r['name']}")

ESC-auto-apt35-ip
ESC-auto-apt28-ip
ESC-auto-apt-c-09-dns
ESC-auto-apt43-ip
unknown_apt_031525_ip
unknown_apt_022025_ip
ESC-auto-apt35-dns
ET_alert_rules_APT_C_48
ESC-auto-apt36-dns
ESC-manual-apt35-ip
ET_alert_rules_APT_C_23
ET_alert_rules_APT10
ET_alert_rules_APT42
ET_alert_rules_APT28
ET_alert_rules_APT32


Создание нового списка

In [48]:
new_list = replists.create_list(
    name="test_list_01",
    type="ip",
    color="1",
    description="Тестовый список IP-адресов",
    content="1.1.1.1\n2.2.2.2"
)
print(f"Список создан: {new_list['id']}")

Список создан: 26144


Обновление списка

In [49]:
updated = replists.update_list(list_id=new_list["id"], description="Обновлённый тестовый список IP-адресов")
print("Описание обновлено:", updated["description"])

Описание обновлено: Обновлённый тестовый список IP-адресов


Удаление списка

In [50]:
replists.delete_list(list_id=new_list["id"])
print("Список удалён")

Список удалён


#### Signatures

Получение всех классов сигнатур

In [77]:
from ptnad import PTNADClient
from ptnad.api.signatures import SignaturesAPI

client = PTNADClient("https://ptlab-nad.ptlab.ptsecurity.ru", verify_ssl=False)
client.set_auth(username="administrator", password="Administr@t0r")
client.login()

signatures = SignaturesAPI(client)

classes = signatures.get_classes()

# Для удобства установлено ограничение на вывод 10 классов
for cls in classes[:10]:
    print(f"{cls['title']} (priority: {cls['priority']})")


Not suspicious traffic (priority: 3)
Unknown traffic (priority: 3)
Potentially bad traffic (priority: 2)
Information leak attempt (priority: 2)
DoS attack attempt (priority: 2)
Attempt to gain user privileges (priority: 2)
Attempt to gain administrator privileges (priority: 2)
RPC query decoding (priority: 2)
Executable code (priority: 1)
Suspicious string (priority: 3)


Получение полной информации по одной сигнатуре

In [45]:
rules = signatures.get_rules(search="ftp", cls="Network scan")

for r in rules:
    print(f"\nRule #{r['sid']} - {r['msg']}")
    print(f"Vendor: {r['vendor']}, Enabled: {r['enabled']}")



Rule #2007802 - ET SCAN Grim's Ping ftp scanning tool
Vendor: ETOpen, Enabled: True
